<img src="https://github.com/LucaScaldaferri/Curso_Cientista_de_Dados/blob/main/logo_ebac.png?raw=true" alt="ebac-logo">


---

# **Profissão Cientista de Dados**
## **Módulo** 13 | Regressão II | Exercício 2
Resolução Caderno de **Exercícios**<br> 
Aluno [Lucas Scaldaferri Magalhães Freitas](https://www.linkedin.com/in/0scaldaferri-lucas/) <br>

---

# EBAC - Regressão II - regressão múltipla

## Tarefa I

#### Previsão de renda II

Vamos continuar trabalhando com a base 'previsao_de_renda.csv', que é a base do seu próximo projeto. Vamos usar os recursos que vimos até aqui nesta base.

|variavel|descrição|
|-|-|
|data_ref                | Data de referência de coleta das variáveis |
|index                   | Código de identificação do cliente|
|sexo                    | Sexo do cliente|
|posse_de_veiculo        | Indica se o cliente possui veículo|
|posse_de_imovel         | Indica se o cliente possui imóvel|
|qtd_filhos              | Quantidade de filhos do cliente|
|tipo_renda              | Tipo de renda do cliente|
|educacao                | Grau de instrução do cliente|
|estado_civil            | Estado civil do cliente|
|tipo_residencia         | Tipo de residência do cliente (própria, alugada etc)|
|idade                   | Idade do cliente|
|tempo_emprego           | Tempo no emprego atual|
|qt_pessoas_residencia   | Quantidade de pessoas que moram na residência|
|renda                   | Renda em reais|

In [59]:
import numpy as np
import pandas as pd

import matplotlib.pyplot as plt
import seaborn as sns

from sklearn.model_selection import train_test_split

import patsy
import statsmodels.api as sm
import statsmodels.formula.api as smf

from sklearn.tree import DecisionTreeRegressor
from sklearn.metrics import r2_score

In [60]:
df = pd.read_csv('previsao_de_renda.csv')

In [61]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 15000 entries, 0 to 14999
Data columns (total 15 columns):
 #   Column                 Non-Null Count  Dtype  
---  ------                 --------------  -----  
 0   Unnamed: 0             15000 non-null  int64  
 1   data_ref               15000 non-null  object 
 2   id_cliente             15000 non-null  int64  
 3   sexo                   15000 non-null  object 
 4   posse_de_veiculo       15000 non-null  bool   
 5   posse_de_imovel        15000 non-null  bool   
 6   qtd_filhos             15000 non-null  int64  
 7   tipo_renda             15000 non-null  object 
 8   educacao               15000 non-null  object 
 9   estado_civil           15000 non-null  object 
 10  tipo_residencia        15000 non-null  object 
 11  idade                  15000 non-null  int64  
 12  tempo_emprego          12427 non-null  float64
 13  qt_pessoas_residencia  15000 non-null  float64
 14  renda                  15000 non-null  float64
dtypes:

1. Separe a base em treinamento e teste (25% para teste, 75% para treinamento).
2. Rode uma regularização *ridge* com alpha = [0, 0.001, 0.005, 0.01, 0.05, 0.1] e avalie o $R^2$ na base de testes. Qual o melhor modelo?
3. Faça o mesmo que no passo 2, com uma regressão *LASSO*. Qual método chega a um melhor resultado?
4. Rode um modelo *stepwise*. Avalie o $R^2$ na vase de testes. Qual o melhor resultado?
5. Compare os parâmetros e avalie eventuais diferenças. Qual modelo você acha o melhor de todos?
6. Partindo dos modelos que você ajustou, tente melhorar o $R^2$ na base de testes. Use a criatividade, veja se consegue inserir alguma transformação ou combinação de variáveis.
7. Ajuste uma árvore de regressão e veja se consegue um $R^2$ melhor com ela.

In [62]:
#1

df_train, df_test = train_test_split(df, test_size=0.25, random_state=1000)

In [73]:
#2 - Regularização ridge

modelo = '''renda ~ sexo 
                    + posse_de_veiculo 
                    + posse_de_imovel 
                    + qtd_filhos 
                    + tipo_renda 
                    + educacao 
                    + estado_civil 
                    + tipo_residencia 
                    + idade 
                    + tempo_emprego 
                    + qt_pessoas_residencia
                    '''
y_train, X_train = patsy.dmatrices(modelo, df_train)
y_test, X_test = patsy.dmatrices(modelo, df_test)

In [84]:
alphas = [0, 0.001, 0.005, 0.01, 0.05, 0.1]
r2_pred = []

for alpha in alphas:
    md = sm.OLS(y_train, X_train)
    reg = md.fit_regularized(method='elastic_net', refit=True, L1_wt=0, alpha=alpha)
    y_pred = reg.predict(X_test)
    r2 = r2_score(y_test, y_pred)
    
    r2_pred.append(r2)
    
pd.DataFrame({'Alpha':alphas, 'R2':r2_pred}).sort_values(by='R2', ascending=False)

,Alpha,R2
0,0.000,0.262434
1,0.001,0.262262
2,0.005,0.261966
3,0.010,0.261568
4,0.050,0.256383
5,0.100,0.248634


Ridge: o modelo que apresentou melhor resultado foi o de alpha = 0, obetando R² = 0.262434

In [85]:
#3 - Regularização lasso

alphas = [0, 0.001, 0.005, 0.01, 0.05, 0.1]
r2_pred = []

for alpha in alphas:
    md = sm.OLS(y_train, X_train)
    reg = md.fit_regularized(method='elastic_net', refit=True, L1_wt=1, alpha=alpha)
    y_pred = reg.predict(X_test)
    r2 = r2_score(y_test, y_pred)
    
    r2_pred.append(r2)
    
pd.DataFrame({'Alpha':alphas, 'R2':r2_pred}).sort_values(by='R2', ascending=False)

,Alpha,R2
4,0.050,0.262500
0,0.000,0.262434
1,0.001,0.262434
2,0.005,0.262434
3,0.010,0.262434
5,0.100,0.262434


Lasso: o modelo que apresentou melhor resultado foi o de alpha = 0.050, obetando R² = 0.262500

In [89]:
#4 Stepwise

df_dummies = pd.get_dummies(data=df.dropna(), drop_first=True).astype(int)

X = df_dummies.drop(columns='renda')
y = df_dummies['renda']

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.25, random_state=1000)

In [90]:
def stepwise_selection(X, y, 
                       initial_list=[], 
                       threshold_in=0.05, 
                       threshold_out = 0.05, 
                       verbose=True):
    """ Perform a forward-backward feature selection 
    based on p-value from statsmodels.api.OLS
    Arguments:
        X - pandas.DataFrame with candidate features
        y - list-like with the target
        initial_list - list of features to start with (column names of X)
        threshold_in - include a feature if its p-value < threshold_in
        threshold_out - exclude a feature if its p-value > threshold_out
        verbose - whether to print the sequence of inclusions and exclusions
    Returns: list of selected features 
    Always set threshold_in < threshold_out to avoid infinite looping.
    See https://en.wikipedia.org/wiki/Stepwise_regression for the details
    """
    included = list(initial_list)
    while True:
        changed=False
        
        # forward step
        excluded = list(set(X.columns)-set(included))
        new_pval = pd.Series(index=excluded, dtype=np.dtype('float64'))
        for new_column in excluded:
            model = sm.OLS(y, sm.add_constant(pd.DataFrame(X[included+[new_column]]))).fit()
            new_pval[new_column] = model.pvalues[new_column]
        best_pval = new_pval.min()
        if best_pval < threshold_in:
            best_feature = new_pval.index[new_pval.argmin()]
            included.append(best_feature)
            changed=True
            if verbose:
                 print('Add  {:30} with p-value {:.6}'.format(best_feature, best_pval))

        # backward step
        print("#############")
        print(included)
        model = sm.OLS(y, sm.add_constant(pd.DataFrame(X[included]))).fit()
        # use all coefs except intercept
        pvalues = model.pvalues.iloc[1:]
        worst_pval = pvalues.max() # null if pvalues is empty
        if worst_pval > threshold_out:
            changed=True
            worst_feature = pvalues.argmax()
            included.remove(worst_feature)
            if verbose:
                print('Drop {:30} with p-value {:.6}'.format(worst_feature, worst_pval))
        if not changed:
            break
    return included

variaveis = stepwise_selection(X_test, y_test)

print('\n\nresulting features:')
print(variaveis)

Add  tempo_emprego                  with p-value 1.94707e-106
#############
['tempo_emprego']
Add  sexo_M                         with p-value 7.62337e-99
#############
['tempo_emprego', 'sexo_M']
Add  educacao_Superior completo     with p-value 0.00626981
#############
['tempo_emprego', 'sexo_M', 'educacao_Superior completo']
Add  data_ref_2015-05-01            with p-value 0.0107069
#############
['tempo_emprego', 'sexo_M', 'educacao_Superior completo', 'data_ref_2015-05-01']
Add  tipo_renda_Empresário          with p-value 0.0173589
#############
['tempo_emprego', 'sexo_M', 'educacao_Superior completo', 'data_ref_2015-05-01', 'tipo_renda_Empresário']
Add  idade                          with p-value 0.0181002
#############
['tempo_emprego', 'sexo_M', 'educacao_Superior completo', 'data_ref_2015-05-01', 'tipo_renda_Empresário', 'idade']
Add  data_ref_2015-04-01            with p-value 0.0307243
#############
['tempo_emprego', 'sexo_M', 'educacao_Superior completo', 'data_ref_2015-05-0

In [92]:
X = df_dummies[variaveis]
y = df_dummies['renda']
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.25, random_state=1000)

y_pred = sm.OLS(y_train, X_train).fit().predict(X_test)
print(f'R2: {r2_score(y_test, y_pred)}')

R2: 0.25900505277245334


In [94]:
# Regularização ridge

alphas = [0, 0.001, 0.005, 0.01, 0.05, 0.1]
r2_pred = []

for alpha in alphas:
    md = sm.OLS(y_train, X_train)
    reg = md.fit_regularized(method='elastic_net', refit=True, L1_wt=0, alpha=alpha)
    y_pred = reg.predict(X_test)
    r2 = r2_score(y_test, y_pred)
    
    r2_pred.append(r2)
    
pd.DataFrame({'Alpha':alphas, 'R2':r2_pred}).sort_values(by='R2', ascending=False)

,Alpha,R2
1,0.001,0.259011
0,0.000,0.259005
2,0.005,0.258996
3,0.010,0.258895
4,0.050,0.255851
5,0.100,0.249368


In [96]:
# Regularização lasso

alphas = [0, 0.001, 0.005, 0.01, 0.05, 0.1]
r2_pred = []

for alpha in alphas:
    md = sm.OLS(y_train, X_train)
    reg = md.fit_regularized(method='elastic_net', refit=True, L1_wt=1, alpha=alpha)
    y_pred = reg.predict(X_test)
    r2 = r2_score(y_test, y_pred)
    
    r2_pred.append(r2)
    
pd.DataFrame({'Alpha':alphas, 'R2':r2_pred}).sort_values(by='R2', ascending=False)

,Alpha,R2
0,0.000,0.259005
1,0.001,0.259005
2,0.005,0.259005
3,0.010,0.259005
4,0.050,0.259003
5,0.100,0.259003


Ambos os modelos são muito parecidos, todavia, o melhor modelo utilizou regressão de regularização ridge com alpha = 0.001 e R² = 0.259011.
Porém, a regressão de regularização lasso apresentou valores semelhantes de R² para diferentes alphas.

#5

Dentre todos os modelos utilizados, o que apresentou melhor R², ou seja, o melhor modelo a ser considerado, foi o 2º modelo utilizando reguralização lasso (R² = 0.262500)

In [98]:
#6

modelo = '''np.log(renda) ~ sexo 
                        + posse_de_veiculo 
                        + posse_de_imovel 
                        + tipo_renda == 'Empresário'
                        + tipo_renda == 'Servidor público'
                        + educacao == 'Superior completo'
                        + idade 
                        + tempo_emprego 
                        + qt_pessoas_residencia == 3
                        + qt_pessoas_residencia == 4
                        '''
y_train, X_train = patsy.dmatrices(modelo, df_train)
y_test, X_test = patsy.dmatrices(modelo, df_test)

alphas = [0, 0.001, 0.005, 0.01, 0.05, 0.1]
r2_pred = []

for alpha in alphas:
    md = sm.OLS(y_train, X_train)
    reg = md.fit_regularized(method='elastic_net', refit=True, L1_wt=1, alpha=alpha)
    y_pred = reg.predict(X_test)
    r2 = r2_score(y_test, y_pred)
    
    r2_pred.append(r2)
    
pd.DataFrame({'Alpha':alphas, 'R2':r2_pred}).sort_values(by='R2', ascending=False)

,Alpha,R2
0,0.000,0.359882
1,0.001,0.355790
3,0.010,0.349849
2,0.005,0.349336
4,0.050,0.349336
5,0.100,0.349336


Utilizando a mesma base do modelo do exercício anterior, o R² do modelo deu um salto de 0.262500 para 0.359882, um ganho de 9,7382%. <br>
Desta forma, este modelo explica quase 10% melhor a variação na resposta.

In [107]:
#7

reg_tree = DecisionTreeRegressor(random_state=1000, max_depth=5, min_samples_leaf=13)
reg_tree.fit(X_train, y_train)

DecisionTreeRegressor(max_depth=5, min_samples_leaf=13, random_state=1000)

In [108]:
print('R2 :', 
      reg_tree.score(X=X_test, y=y_test))

R2 : 0.3904544875108438


Árvore de regressão apresentou um resultado ainda melhor.